# Hidden Markov Models

### Problem Statement 
The following problem is from the Udacity course on Artificial Intelligence (Thrun and Norvig), chapter 11 (HMMs and filters). It involves a simple scenario where a person's current emotional state is determined by the weather on that particular day. The task is to find the underlying hidden sequence of states (in this case, the weather), given only a set of observations (moods) and information about state/observation changes.

In [1]:
#import required libraries
import numpy as np
import warnings
from pprint import pprint

P(Rainy weather) = P(R) = 0.5  (initial probabilites)  
P(Sunny weather) = P(S) = 0.5  

The chances of weather changing are given as 40%, so  
P(S->R) i.e P( R-today | S-yesterday ) = 0.4, therefore P(S->S) i.e P( S-today | S-yesterday ) = 0.6  
P(R->S) i.e P( S-today | R-yesterday ) = 0.4, and P(R->R) i.e P( R-today | R-yesterday ) = 0.6  
For the purpose of formulating an HMM, we call the above ***Transition Probabilities.*** 

The corresponding mood changes, given the weather are :  
P(R->H) i.e P( H | R ) = 0.4, therefore P( G | R ) = 0.6  
P(S->H) i.e P( H | S ) = 0.9, and P( G | S ) = 0.1  
We call these ***Emission Probabilities***

In [2]:
S = np.array([0, 1]) # 0 Rainy, 1 Sunny
S_names = ('Rainy', 'Sunny') 
pi = np.array([0.5, 0.5]) # Initial Probabilities
O = np.array(['Happy', 'Grumpy']) # Set of observations
A = np.array([[0.6, 0.4], [0.2, 0.8]]) # {R:{R, S}, S:{R, S}} Transition Matrix
B = np.array([[0.4, 0.6], [0.9, 0.1]]) # {R: {H, G}, S: {H, G}} Emission Matrix
Y = np.array([0, 0, 1]) # 0 Happy, 1 Grumpy -- Observation sequence

### Hidden Markov Models

[HMM](https://en.wikipedia.org/wiki/Hidden_Markov_model)s are a class of probabilistic graphical models that can predict the sequence of states, given a sequence of observations that are dependent on those states. HMMs have seen widespread success in a variety of applications, ranging from Robotics to Speech Processing and Translation. An HMM operates according to a set of assumptions, which are :  
1. ** Markov Assumption **  
Current state is dependent on only the previous state.  
2. ** Stationarity Assumption **  
Transition probabilities are independent of time of transition.  
3. ** Independence Assumption **  
Each observation depends solely on the current underlying state (which in turn depends on the previous one), and is independent of other observations.  

An HMM is a **Generative model**, in that it attempts to find the probability of a set of observations being produced or *generated* by a class.  The parameters that we pass to the HMM class, defined below, are:  
*O* = a set of observations  
*S* = a set of states  
*A* = transition probabilities, represented as a matrix  
*B* = emission probabilities, represented as a matrix  
*pi* = initial state probabilties  
*Y* = sequence observed  

In [3]:
class HMM:
    def __init__(self, observations, states, start_probs, trans_probs, emm_probs , obs_sequence):
        self.O = observations
        self.S = states
        self.pi = start_probs
        self.A = trans_probs
        self.B = emm_probs
        self.Y = obs_sequence
        self.k = self.S.shape[0]
        self.t = self.Y.shape[0]
        self.table_1 = np.zeros((self.k, self.t))
        self.table_2 = np.zeros((self.k, self.t))
        self.output_sequence = np.zeros((self.t,))
        self.fwds = None
        self.bwds = None
        self.smoothened = None
        
        
    def viterbi(self):
        # loop through states, but only for first observation
        for i in range(self.k):
            print "Day 1 for state", i
            self.table_1[i, 0] = self.pi[i] * self.B[i, self.Y[0]]
            print "Table 1 entry set to", i, "-->", self.table_1[i, 0]
            self.table_2[i, 0] = 0
            print "Table 2 entry set to ->", self.table_2[i, 0]
            print "-------------------------------------------"
        print "========================================="
        # loop through second to last observation 
        for i in range(1, self.t):
            for j in range(self.k): # loop through states
                print "Day", i+1, "for state ", j
                max_t1_A = 0.0
                argmax_t1_A = 0
                for d in range(self.k): # loop through states*states
                    val = self.table_1[d, i-1]*self.A[d, j]
                    print "state", d, "from state", j, "-->", val
                    if val > max_t1_A:
                        max_t1_A = val
                        argmax_t1_A = d
                    else:
                        continue
                print "Max was state", argmax_t1_A, "-->", max_t1_A
                self.table_1[j, i] = max_t1_A
                tmp = self.table_1[j, i]
                self.table_1[j, i] = self.table_1[j, i]*self.B[j, self.Y[i]]
                print "Table 1 entry set to -->", tmp, "*", self.B[j, self.Y[i]], " = ", self.table_1[j, i]
                self.table_2[j, i] = argmax_t1_A
                print "Table 2 entry set to ->", self.table_2[j, i]
                print "-------------------------------------------"
            print "==========================================="
        
        max_t1 = 0.0
        argmax_t1 = 0
        for d in range(self.k):
            val = self.table_1[d, self.t-1]
            if val > max_t1:
                max_t1 = val
                argmax_t1 = d
            else:
                continue
        z = np.zeros((self.t, ))
        z[-1] = argmax_t1
        print "==========================================="
        print "Day", self.t, "prob was highest for state", int(z[-1]), "-->", max_t1
        self.output_sequence[-1] = self.S[int(z[-1])] # Maps final state to sequence first
        for i in range(self.t-1, 0, -1): # For days from next-last to first
            z[i-1] = self.table_2[int(z[i]), i]
            print "Day", i, "prob was highest for state", int(z[i-1]), "-->", self.table_1[int(z[i-1]), i]
            self.output_sequence[i-1] = self.S[int(z[i-1])]
        return self.output_sequence
    
    def get_obs(self, obs_val, emm_prob):
        ob_mat = np.zeros((self.k, self.k))
        for i in self.S:
            for j in self.S:
                if i == j:
                    ob_mat[i, j] = emm_prob[i, obs_val]
        return ob_mat
        
    def get_diagonal(self, mat_A, mat_B):
        x = np.transpose(mat_A).shape[1]
        mat_C = np.dot(mat_A, np.transpose(mat_B))
        mat_D = np.zeros((self.k, 1))
        for i in range(x):
            for j in range(x):
                if i==j:
                    mat_D[i][0] = mat_C[i][j]
        return mat_D
        
        
    def forward_backward(self):
        self.m = self.O.shape[0]
        #print self.m
        obs_mats = [None for i in range(self.t)]
        for i in range(self.t):
            obs_mats[i] = self.get_obs(self.Y[i], self.B)
        
        print "Observation matrices :"
        pprint(obs_mats)
        print ""
        
        #forward probability calculation
        f = [[] for i in range(self.t+1)]
        f[0] = self.pi.reshape(self.k, 1)
        csum = 0.0
        for j in f[0]:
            csum += j
        for j in range(f[0].shape[0]):
            f[0][j] = f[0][j]/csum
        for i in range(1, self.t+1):
            #print "obs", obs_mats[i-1]
            #print "prev f", f[i-1]
            f[i] = np.dot(np.dot(obs_mats[i-1], self.A), f[i-1]).reshape(self.k, 1)
            #scaling done here
            csum = 0.0
            for j in f[i]:
                csum += j
            for j in range(f[i].shape[0]):
                f[i][j] = f[i][j]/csum
            #print "new f", f[i]
        f = np.array(f)
        print "Forward probabilities :"
        pprint(f)
        print ""
        
        #backward probability calculation
        b = [[] for i in range(self.t+1)]
        b[-1] = np.array([[1.0] for i in range(self.k)])
        for i in range(self.t-1, -1, -1):
            b[i] = np.dot(np.dot(self.A, obs_mats[i]), b[i+1]).reshape(self.k, 1)
            #scaling done here
            csum = 0.0
            for j in b[i]:
                csum += j
            for j in range(b[i].shape[0]):
                b[i][j] = b[i][j]/csum
        b = np.array(b)
        print "Backward probabilities :"
        pprint(b)
        print ""
        
        #smoothed values
        smooth = [[] for i in range(self.t+1)]
        for i in range(self.t+1):
            smooth[i] = self.get_diagonal(f[i], b[i])
            csum = 0.0
            for j in smooth[i]:
                csum += j
            for j in range(smooth[i].shape[0]):
                smooth[i][j] = smooth[i][j]/csum
        smooth = np.array(smooth)
        print "Smoothed probabilities :"
        pprint(smooth)
        
        self.fwds = f
        self.bwds = b
        self.smoothened = smooth
        for i in range(1, smooth.shape[0]):
            max_prob = max(smooth[i].tolist())
            print "Day", i, "probability was max for state", smooth[i].tolist().index(max_prob), "-->", max_prob[0]
            self.output_sequence[i-1] = smooth[i].tolist().index(max_prob)
        return self.output_sequence

### Viterbi Algorithm

Explanation : **TO-DO**

In [4]:
weather_hmm = HMM(O, S, pi, A, B, Y)
obs_states = [O[i] for i in Y]
print "Observations :"
print obs_states, "\n"
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print "Using Viterbi Algorithm:"
    op1 = weather_hmm.viterbi()
    op_states1 = [S_names[int(i)] for i in op1]
print op_states1

Observations :
['Happy', 'Happy', 'Grumpy'] 

Using Viterbi Algorithm:
Day 1 for state 0
Table 1 entry set to 0 --> 0.2
Table 2 entry set to -> 0.0
-------------------------------------------
Day 1 for state 1
Table 1 entry set to 1 --> 0.45
Table 2 entry set to -> 0.0
-------------------------------------------
Day 2 for state  0
state 0 from state 0 --> 0.12
state 1 from state 0 --> 0.09
Max was state 0 --> 0.12
Table 1 entry set to --> 0.12 * 0.4  =  0.048
Table 2 entry set to -> 0.0
-------------------------------------------
Day 2 for state  1
state 0 from state 1 --> 0.08
state 1 from state 1 --> 0.36
Max was state 1 --> 0.36
Table 1 entry set to --> 0.36 * 0.9  =  0.324
Table 2 entry set to -> 1.0
-------------------------------------------
Day 3 for state  0
state 0 from state 0 --> 0.0288
state 1 from state 0 --> 0.0648
Max was state 1 --> 0.0648
Table 1 entry set to --> 0.0648 * 0.6  =  0.03888
Table 2 entry set to -> 1.0
-------------------------------------------
Day 3 for 

### Forward-Backward Algorithm

Explanation : **TO-DO**

In [8]:
#reset output sequence values to zero
weather_hmm.output_sequence = np.zeros((weather_hmm.t,))
print "Using Forward-Backward Algorithm:"
op2 = weather_hmm.forward_backward()
op_states2 = [S_names[int(i)] for i in op2]
print op_states2

Using Forward-Backward Algorithm:
Observation matrices :
[array([[ 0.4,  0. ],
       [ 0. ,  0.9]]),
 array([[ 0.4,  0. ],
       [ 0. ,  0.9]]),
 array([[ 0.6,  0. ],
       [ 0. ,  0.1]])]

Forward probabilities :
array([[[ 0.5       ],
        [ 0.5       ]],

       [[ 0.30769231],
        [ 0.69230769]],

       [[ 0.25      ],
        [ 0.75      ]],

       [[ 0.80597015],
        [ 0.19402985]]])

Backward probabilities :
array([[[ 0.42519685],
        [ 0.57480315]],

       [[ 0.48837209],
        [ 0.51162791]],

       [[ 0.66666667],
        [ 0.33333333]],

       [[ 1.        ],
        [ 1.        ]]])

Smoothed probabilities :
array([[[ 0.42519685],
        [ 0.57480315]],

       [[ 0.29787234],
        [ 0.70212766]],

       [[ 0.4       ],
        [ 0.6       ]],

       [[ 0.80597015],
        [ 0.19402985]]])
Day 1 probability was max for state 1 --> 0.702127659574
Day 2 probability was max for state 1 --> 0.6
Day 3 probability was max for state 0 --> 0.80597014